In [1]:
import cv2
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from shapely.geometry import box
import os
import random

matplotlib.use('TkAgg')  # 或者 'Qt5Agg'




In [2]:
def load_classes(classes_file):
    """加载标签映射"""
    with open(classes_file, 'r') as file:
        classes = [line.strip() for line in file.readlines()]
    class_to_id = {name: idx for idx, name in enumerate(classes)}
    id_to_class = {idx: name for idx, name in enumerate(classes)}
    return class_to_id, id_to_class

def read_yolo_labels_with_text(label_path, id_to_class, img_width, img_height):
    """读取 YOLO 格式标签并转换为文本标签"""
    labels = []
    with open(label_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            class_id = int(parts[0])
            x_center, y_center, width, height = map(float, parts[1:5])
            class_name = id_to_class[class_id]  # 转换为文本标签
            labels.append((class_name, x_center, y_center, width, height))
    return labels

def save_yolo_labels_with_text(labels, save_path, class_to_id, img_width, img_height):
    """保存 YOLO 格式标签，文本标签转换回数值"""
    with open(save_path, 'w') as file:
        for label in labels:
            class_name, x_center, y_center, width, height = label
            class_id = class_to_id[class_name]  # 转换为数值标签
            file.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

def yolo_to_pixel_with_text(label, img_width, img_height):
    """将 YOLO 格式转换为像素坐标并使用文本标签"""
    class_name, x_center, y_center, width, height = label
    x_center *= img_width
    y_center *= img_height
    width *= img_width
    height *= img_height
    x_min = int(x_center - width / 2)
    y_min = int(y_center - height / 2)
    x_max = int(x_center + width / 2)
    y_max = int(y_center + height / 2)
    return class_name, x_min, y_min, x_max, y_max

def modify_label_by_index(labels, index, new_class_name):
    """根据索引修改标签类别"""
    if 0 <= index < len(labels):
        old_label = labels[index][0]  # 获取原类别名称
        labels[index] = (new_class_name, *labels[index][1:])  # 替换类别名称
        print(f"索引 {index} 的标签已从 '{old_label}' 修改为 '{new_class_name}'")
    else:
        raise IndexError(f"无效的索引: {index}")



In [3]:
def plot_image_with_boxes(img_path, labels, img_width, img_height):
    """显示带有边界框和文本标签的图像，同时打印索引和标签信息"""
    # 读取图像
    img = cv2.imread(img_path)
    if img is None:
        raise ValueError(f"无法加载图像: {img_path}")
    
    # 转换为 RGB 格式
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # 创建绘图
    fig, ax = plt.subplots(1, figsize=(12, 8))
    ax.imshow(img)

    print("标签信息 (索引, 类别, 中心坐标, 宽度, 高度):")
    for i, label in enumerate(labels):
        # 转换 YOLO 格式为像素坐标
        class_name, x_min, y_min, x_max, y_max = yolo_to_pixel_with_text(label, img_width, img_height)

        # 绘制边界框
        rect = Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)

        random_color = (random.random(), random.random(), random.random())

        # 显示标签文本
        ax.text(x_min, y_min - 5, f"{i}:{class_name}", color=random_color, fontsize=16)

        # 打印索引和标签信息
        print(f"{i}. 类别: {class_name}, 中心: ({label[1]:.2f}, {label[2]:.2f}), 宽度: {label[3]:.2f}, 高度: {label[4]:.2f}")

    plt.show()

In [8]:
file_name = '000126'

In [ ]:
# 设置图像和标签的路径
img_path = 'C:/Users/tfgmo/OneDrive - Virginia Tech/Research/Mahjong CV/cleaned_data_Fall2024/'
label_path = f'C:/Users/tfgmo/OneDrive - Virginia Tech/Research/Mahjong CV/cleaned_data_Fall2024/predicted/predict{int(file_name)}/labels/'
classes_file = 'C:/Users/tfgmo/OneDrive - Virginia Tech/Research/Mahjong CV/cleaned_data_Fall2024/predicted/classes.txt'
class_to_id, id_to_class = load_classes(classes_file)


# 读取图像
image_path = os.path.join(img_path, file_name + '.png')

# 读取 YOLO 格式的标签
label_file_path = os.path.join(label_path, file_name + '.txt')

# 读取图像和标签
img = cv2.imread(image_path)
img_height, img_width, _ = img.shape
labels = read_yolo_labels_with_text(label_file_path, id_to_class, img_width, img_height)

# 显示原始图像和标签
plot_image_with_boxes(image_path, labels, img_width, img_height)



标签信息 (索引, 类别, 中心坐标, 宽度, 高度):
0. 类别: UNKNOWN, 中心: (0.31, 0.90), 宽度: 0.03, 高度: 0.08
1. 类别: UNKNOWN, 中心: (0.28, 0.90), 宽度: 0.03, 高度: 0.07
2. 类别: 2s, 中心: (0.41, 0.18), 宽度: 0.04, 高度: 0.04
3. 类别: 3p, 中心: (0.15, 0.89), 宽度: 0.05, 高度: 0.11
4. 类别: 6s, 中心: (0.52, 0.51), 宽度: 0.04, 高度: 0.08
5. 类别: 7p, 中心: (0.16, 0.38), 宽度: 0.04, 高度: 0.08
6. 类别: 1s, 中心: (0.56, 0.52), 宽度: 0.04, 高度: 0.09
7. 类别: 8p, 中心: (0.27, 0.31), 宽度: 0.03, 高度: 0.07
8. 类别: 7p, 中心: (0.46, 0.29), 宽度: 0.05, 高度: 0.06
9. 类别: 6s, 中心: (0.17, 0.30), 宽度: 0.04, 高度: 0.07
10. 类别: 1z, 中心: (0.31, 0.32), 宽度: 0.03, 高度: 0.07
11. 类别: 2p, 中心: (0.10, 0.89), 宽度: 0.05, 高度: 0.12
12. 类别: 5p, 中心: (0.24, 0.89), 宽度: 0.05, 高度: 0.11
13. 类别: 3s, 中心: (0.35, 0.32), 宽度: 0.04, 高度: 0.07
14. 类别: 4p, 中心: (0.20, 0.89), 宽度: 0.05, 高度: 0.11
15. 类别: 6s, 中心: (0.45, 0.10), 宽度: 0.04, 高度: 0.05
16. 类别: 2s, 中心: (0.41, 0.22), 宽度: 0.04, 高度: 0.05
17. 类别: 1p, 中心: (0.24, 0.31), 宽度: 0.03, 高度: 0.06
18. 类别: 8p, 中心: (0.46, 0.24), 宽度: 0.05, 高度: 0.05
19. 类别: 7z, 中心: (0.97, 0.88), 宽度: 0.06, 

In [ ]:
modify_label_by_index(labels, index=12, new_class_name='0p')
# modify_label_by_index(labels, index=9, new_class_name='UNKNOWN')

# modify_label_by_index(labels, index=22, new_class_name='1s')
# modify_label_by_index(labels, index=39, new_class_name='1s')
# modify_label_by_index(labels, index=9, new_class_name='1s')
# modify_label_by_index(labels, index=18, new_class_name='1p')
# modify_label_by_index(labels, index=27, new_class_name='1z')
# modify_label_by_index(labels, index=31, new_class_name='1z')
# modify_label_by_index(labels, index=41, new_class_name='1m')
# modify_label_by_index(labels, index=18, new_class_name='1m')

# modify_label_by_index(labels, index=36, new_class_name='2m')
# modify_label_by_index(labels, index=35, new_class_name='2m')
# modify_label_by_index(labels, index=54, new_class_name='2p')
# modify_label_by_index(labels, index=35, new_class_name='2s')
# modify_label_by_index(labels, index=34, new_class_name='2z')
# modify_label_by_index(labels, index=35, new_class_name='2z')

# modify_label_by_index(labels, index=16, new_class_name='3s')
# modify_label_by_index(labels, index=22, new_class_name='3m')
# # modify_label_by_index(labels, index=35, new_class_name='3p')
# modify_label_by_index(labels, index=6, new_class_name='3z')
# modify_label_by_index(labels, index=25, new_class_name='3z')
# modify_label_by_index(labels, index=21, new_class_name='3z')
# modify_label_by_index(labels, index=32, new_class_name='3z')

# modify_label_by_index(labels, index=50, new_class_name='4m')
# modify_label_by_index(labels, index=21, new_class_name='4p')
# modify_label_by_index(labels, index=37, new_class_name='4s')
# modify_label_by_index(labels, index=29, new_class_name='4z')

# modify_label_by_index(labels, index=13, new_class_name='5p')
# # modify_label_by_index(labels, index=25, new_class_name='5s')
# modify_label_by_index(labels, index=33, new_class_name='5s')
# modify_label_by_index(labels, index=47, new_class_name='5m')
# modify_label_by_index(labels, index=28, new_class_name='5m')

# modify_label_by_index(labels, index=32, new_class_name='6m')
# modify_label_by_index(labels, index=35, new_class_name='6m')
# modify_label_by_index(labels, index=33, new_class_name='6m')
# modify_label_by_index(labels, index=23, new_class_name='6z')
# modify_label_by_index(labels, index=66, new_class_name='6z')
# modify_label_by_index(labels, index=57, new_class_name='6z')
# modify_label_by_index(labels, index=27, new_class_name='6s')
# modify_label_by_index(labels, index=31, new_class_name='6p')

# modify_label_by_index(labels, index=1, new_class_name='7p')
# modify_label_by_index(labels, index=0, new_class_name='7p')
# modify_label_by_index(labels, index=6, new_class_name='7p')
# modify_label_by_index(labels, index=18, new_class_name='7s')
# modify_label_by_index(labels, index=24, new_class_name='7m')
# modify_label_by_index(labels, index=18, new_class_name='7m')
# modify_label_by_index(labels, index=16, new_class_name='7m')
# modify_label_by_index(labels, index=19, new_class_name='7z')

# modify_label_by_index(labels, index=27, new_class_name='8s')
# modify_label_by_index(labels, index=24, new_class_name='8m')
# modify_label_by_index(labels, index=38, new_class_name='8m')
# modify_label_by_index(labels, index=9, new_class_name='8p')

# modify_label_by_index(labels, index=20, new_class_name='9s')
# modify_label_by_index(labels, index=19, new_class_name='9s')
# modify_label_by_index(labels, index=15, new_class_name='9m')
# modify_label_by_index(labels, index=11, new_class_name='9m')
# modify_label_by_index(labels, index=42, new_class_name='9p')

In [7]:

# 保存修改后的标签
save_yolo_labels_with_text(labels, label_file_path, class_to_id, img_width, img_height)


# # 显示调整后的图像和标签
# plot_image_with_boxes(image_path, labels, img_width, img_height)